# 数据集介绍

数据集类型是微博情感分类 

来源https://github.com/SophonPlus/ChineseNlpCorpus/blob/master/datasets/weibo_senti_100k/intro.ipynb

In [1]:
import sys,os
import pandas as pd
import random
import numpy as np

In [2]:
module_path=os.path.abspath('/root/NLP_warehouse/reconstruct/basictask/')
if module_path not in sys.path:
    sys.path.append(module_path)
print(sys.path)

['', '/root/miniconda3/envs/nlp_sr/lib/python36.zip', '/root/miniconda3/envs/nlp_sr/lib/python3.6', '/root/miniconda3/envs/nlp_sr/lib/python3.6/lib-dynload', '/root/.local/lib/python3.6/site-packages', '/root/miniconda3/envs/nlp_sr/lib/python3.6/site-packages', '/root/miniconda3/envs/nlp_sr/lib/python3.6/site-packages/IPython/extensions', '/root/.ipython', '/root/NLP_warehouse/reconstruct/basictask']


In [4]:
from tasks import cls
from evaluation import clsEvaluator

In [4]:
data_path='../../datasets/weibo_senti/weibo_senti_100k.csv'
pd_all = pd.read_csv(data_path)

print('评论数目（总体）：%d' % pd_all.shape[0])
print('评论数目（正向）：%d' % pd_all[pd_all.label==1].shape[0])
print('评论数目（负向）：%d' % pd_all[pd_all.label==0].shape[0])

评论数目（总体）：119988
评论数目（正向）：59993
评论数目（负向）：59995


In [5]:
pd_all.sample(20)

,label,review
107840,0,吃完饭之后吐最近都已经习惯成自然了，我真心是控制不了自己的身体了，想要怎样？！要么给个痛快，...
77002,0,错过了直播回放了下春晚，多牛B的舞台，多欢乐的云迪和力宏那孩子，费大妈虽说像带个假面具但宝刀...
85615,0,知道我现在多困么！可是我根本不敢睡！[抓狂][抓狂][泪][泪]
68304,0,转 //@王权锋:帮转！? //@王浩908: [泪] //@360陶伟华:帮转，最恨这种...
49611,1,周四，是每个女生的幸运日[鼓掌]，当然你要点了这个链接http://t.cn/zl82S2E...
1822,1,安贞180大退房，客人非常有秩序的自觉排队退房，2点之前所有的退房全部退完，非常给力，HOL...
57780,1,#百变大咖秀第五季# 马年马上到，大白组合继续挑战“非人类”，双马贺新年了！人们说，马上能放...
48351,1,#高级定制# cartier定制系列...18k白金的love系列手镯也是那么的被吸引.....
85435,0,狠心的护士小姐?居然还是扎在了昨天输液的血管上！没有天理！还肿了[泪] 我在:http://...
90766,0,腹肌!!!!舔屏ing!!!!//@吉本老师求同行去夏威夷: 那?候的腹肌啊~~[抓狂]@n...


In [6]:
from readers.cls import getExamplesFromData
print(len(pd_all))
random_idx=np.random.permutation(len(pd_all))

119988


In [7]:
sentences=pd_all['review'].values[random_idx].tolist()[:20000]
labels=pd_all['label'].values[random_idx].tolist()[:20000]
print(len(sentences),len(labels))
random_idx=np.random.permutation(len(sentences))

20000 20000


In [8]:
label2id={'0':0,'1':1}
dev_ratio=0.2
dev_nums=int(len(sentences)*dev_ratio)
train_nums=len(sentences)-dev_nums
print(dev_nums)
train_sentences=sentences[:train_nums]
train_labels=labels[:train_nums]
dev_sentences=sentences[-dev_nums:]
dev_labels=labels[-dev_nums:]
train_examples,max_seq_len=getExamplesFromData(sentences=train_sentences,labels=train_labels,label2id=label2id,mode='train',return_max_len=True)
dev_examples=getExamplesFromData(sentences=dev_sentences,labels=dev_labels,label2id=label2id,mode='dev')

2021-08-14 13:27:31 - INFO - getExamplesFromData - 125 : *****************************Logging some train examples*****************************
2021-08-14 13:27:31 - INFO - getExamplesFromData - 126 : Total train nums is : 16000
2021-08-14 13:27:31 - INFO - getExamplesFromData - 129 : 
荨麻疹及其汹涌的来找我，姐快Hold不住了，求良药！求良方！[泪]	0
2021-08-14 13:27:31 - INFO - getExamplesFromData - 129 : 
我很成功地一次又一次躲避了镜头，哈哈。@安徽荣耀 @合肥虎哥 @雒尘LC [哈哈]	1
2021-08-14 13:27:31 - INFO - getExamplesFromData - 129 : 
庆幸的是，咱的西红柿稳赢，我自豪！！！[嘻嘻]//@TIEasy:[鼓掌]	1
2021-08-14 13:27:31 - INFO - getExamplesFromData - 129 : 
@晴晴_apple @可能的世界C小窝 @m265 [泪]//@水瓶座蜜语: [抓狂]	0
2021-08-14 13:27:31 - INFO - getExamplesFromData - 129 : 
你们丫愿意讲[哈哈] //@二逼瓦西里:[泪][泪][泪][泪]	0
2021-08-14 13:27:31 - INFO - getExamplesFromData - 125 : *****************************Logging some dev examples*****************************
2021-08-14 13:27:31 - INFO - getExamplesFromData - 126 : Total dev nums is : 4000
2021-08-14 13:27:31 - INFO - getExamplesFromData - 129 : 


4000


In [9]:
print(sum(train_labels),sum(dev_labels))

8005 2066


In [10]:
model_path='/data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/'
print(max_seq_len)
max_seq_len=min(512,max_seq_len)
cls_model=cls(model_path=model_path,label2id=label2id,max_seq_length=max_seq_len,device='cpu')

2021-08-14 13:27:31 - INFO - __init__ - 50 : The label2id is
 {"0": 0, "1": 1}
2021-08-14 13:27:31 - INFO - __init__ - 58 : Loading model from /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/, which is from huggingface model
2021-08-14 13:27:31 - INFO - get_config_dict - 177 : loading configuration file /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/config.json
2021-08-14 13:27:31 - INFO - from_pretrained - 404 : loading archive file /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/
2021-08-14 13:27:31 - INFO - from_pretrained - 422 : Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "directionality": "bidi",
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddi

225


2021-08-14 13:27:32 - INFO - from_pretrained - 124 : loading vocabulary file /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/vocab.txt


In [11]:
from torch.utils.data import DataLoader
batch_size=32
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

In [12]:
evaluator=clsEvaluator(sentences=dev_sentences,labels=dev_labels,write_csv=False)

2021-08-14 13:27:32 - INFO - __init__ - 30 : Evalautor sentence like : 

2021-08-14 13:27:32 - INFO - __init__ - 32 : UP！虽然你很不和谐//@风言疯语LaiN胖子：为啥你不关注别人，却要别人关注你？学名人啊？[嘻嘻] //@ponponxu：转发微博。	1

2021-08-14 13:27:32 - INFO - __init__ - 32 : @大思萌小火鸟快来啊  //@费猫猫: 下家一号举手，哈哈哈[嘻嘻] //@我家的MONKEY:这伴郎很帅嘛@大思萌小火鸟 婚礼上要看好，小心别人惦记[哈哈]	1

2021-08-14 13:27:32 - INFO - __init__ - 32 : 好感人。。[泪][泪]	0

2021-08-14 13:27:32 - INFO - __init__ - 32 : 恐怕这个公关营销方法在中国行不通吧？[衰]	0

2021-08-14 13:27:32 - INFO - __init__ - 32 : 亲们奉献手指不？ 我来试试[嘻嘻]//@陈涛carlos: [哈哈]可以试试，好玩@安妮宝贝佳丽 @艾子Gio @客家ROCK陈善宝 @吴铭rose	1



In [13]:
cls_model.fit(is_pairs=False,train_dataloader=train_dataloader,evaluator=evaluator,output_path="/data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/")

2021-08-14 13:27:32 - INFO - fit - 99 : 当前是单句子分类任务
2021-08-14 13:27:32 - INFO - fit - 110 : 一个epoch 下，每隔100个step会输出一次loss，每隔250个step会评估一次模型


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

0it [00:00, ?it/s]

2021-08-14 13:27:37 - INFO - __call__ - 48 : ClsEvaluator: Evaluating the model on  dataset in epoch 0 after 1 steps:
2021-08-14 13:27:37 - INFO - predict - 218 : 当前是单句子分类任务预测


  0%|          | 0/125 [00:00<?, ?it/s]

2021-08-14 13:29:08 - INFO - __call__ - 55 : Accuracy: 0.532
2021-08-14 13:29:08 - INFO - __call__ - 62 : AUC: 0.555
2021-08-14 13:29:09 - INFO - save_pretrained - 505 : Model weights saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/pytorch_model.bin
2021-08-14 13:29:09 - INFO - save_pretrained - 150 : Configuration saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/config.json
2021-08-14 13:29:09 - INFO - save_vocab - 51 : Vocab saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/vocab.txt
2021-08-14 13:29:09 - INFO - fit - 185 : In epoch 0, training_step 0, the eval score is 0.5545516567542055, previous eval score is -9999999, model has been saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/
2021-08-14 13:35:01 - INFO - fit - 163 : Epoch : 0, train_step : 100/500, loss_value : 0.3212981130508706 
2021-08-14 13:40:52 - INFO - fit - 163 : Epoch : 0, train_step : 200/500, loss_value : 0.06558689029654488 
2021-08-14 13:43:50 - INFO - __call__ - 48 : ClsEvalu

  0%|          | 0/125 [00:00<?, ?it/s]

2021-08-14 13:45:51 - INFO - __call__ - 55 : Accuracy: 0.981
2021-08-14 13:45:51 - INFO - __call__ - 62 : AUC: 0.998
2021-08-14 13:45:52 - INFO - save_pretrained - 505 : Model weights saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/pytorch_model.bin
2021-08-14 13:45:52 - INFO - save_pretrained - 150 : Configuration saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/config.json
2021-08-14 13:45:52 - INFO - save_vocab - 51 : Vocab saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/vocab.txt
2021-08-14 13:45:52 - INFO - fit - 185 : In epoch 0, training_step 250, the eval score is 0.9981862748533152, previous eval score is 0.5545516567542055, model has been saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/


KeyboardInterrupt: 

In [30]:
really_import=cls(model_path='/data/nfs14/nfs/aisearch/asr/xhsun/tmp_model',
                  label2id=label2id,max_seq_length=max_seq_len,device='cpu',is_finetune=True)

2021-08-14 14:00:06 - INFO - __init__ - 50 : The label2id is
 {"0": 0, "1": 1}
2021-08-14 14:00:06 - INFO - get_config_dict - 177 : loading configuration file /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/config.json
2021-08-14 14:00:06 - INFO - from_pretrained - 404 : loading archive file /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT
2021-08-14 14:00:06 - INFO - from_pretrained - 422 : Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "directionality": "bidi",
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "p

In [31]:
predicts=really_import.predict(is_pairs=False,dataloader=dev_sentences)

2021-08-14 14:01:59 - INFO - predict - 218 : 当前是单句子分类任务预测


  0%|          | 0/125 [00:00<?, ?it/s]

In [37]:
sum((np.argmax(predicts,axis=1)==np.array(dev_labels)).tolist())/len(predicts)

0.981

In [40]:
import torch.nn as nn

In [53]:
class CRF(nn.Module):
    def __init__(self, num_tags, batch_first=True, hasTraining=False) -> None:
        super().__init__()
        self.num_tags = num_tags
        self.batch_first = batch_first

        if hasTraining==False:
            self.start_transitions = nn.Parameter(torch.empty(num_tags))
            self.end_transitions = nn.Parameter(torch.empty(num_tags))
            self.transitions = nn.Parameter(torch.empty(num_tags,num_tags))
            self.init_matrix()
        else:
            pass


    def init_matrix(self):
        nn.init.uniform_(self.start_transitions, -0.1, 0.1)
        nn.init.uniform_(self.end_transitions, -0.1, 0.1)
        nn.init.uniform_(self.transitions, -0.1, 0.1)

In [54]:
crf=CRF(5)

In [56]:
os.path.join("sds",'dsd','ad')

'sds/dsd/ad'

In [51]:
torch.save(obj=crf.transitions,f='./tmp.pt')

In [52]:
torch.load('./tmp.pt')

tensor([[-0.0763,  0.0966, -0.0893, -0.0571, -0.0983, -0.0413,  0.0384,  0.0052,
          0.0677,  0.0197,  0.0469,  0.0046,  0.0783, -0.0790, -0.0673, -0.0206,
         -0.0765, -0.0227,  0.0540, -0.0875],
        [ 0.0077,  0.0380, -0.0522, -0.0449, -0.0687,  0.0733, -0.0563,  0.0222,
         -0.0178, -0.0719,  0.0617, -0.0544,  0.0430,  0.0310, -0.0394, -0.0492,
         -0.0650,  0.0193, -0.0395,  0.0256],
        [-0.0288,  0.0377,  0.0286, -0.0850,  0.0333, -0.0450,  0.0586, -0.0283,
         -0.0705, -0.0768, -0.0528,  0.0722, -0.0189,  0.0898,  0.0761, -0.0595,
          0.0618,  0.0099,  0.0679,  0.0191],
        [-0.0255, -0.0240, -0.0407,  0.0783,  0.0833,  0.0522,  0.0329, -0.0547,
          0.0404,  0.0451,  0.0487, -0.0752, -0.0327, -0.0162, -0.0277, -0.0892,
         -0.0172, -0.0717,  0.0907,  0.0358],
        [-0.0634,  0.0128, -0.0769,  0.0997, -0.0183, -0.0625, -0.0745, -0.0118,
          0.0705, -0.0982, -0.0611,  0.0490,  0.0317, -0.0486, -0.0417,  0.0550,
      